# Train and evaluate a PC

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

Set the random seeds.

In [ ]:
random.seed(42)
np.random.seed(42)

## Load MNIST Dataset

Load the training and test splits of MNIST, and preprocess them by flattening the tensor images.

In [ ]:
from torchvision import transforms, datasets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: (255 * x.view(-1)).long())
])
data_train = datasets.MNIST('datasets', train=True, download=True, transform=transform)
data_test = datasets.MNIST('datasets', train=False, download=True, transform=transform)
num_variables = data_train[0][0].shape[0]
height, width = 28, 28
print(f"Number of variables: {num_variables}")

In [ ]:
plt.matshow(data_train[0][0].reshape(28, 28), cmap='gray')
plt.title(f"Class: {data_train[0][1]}")
plt.show()

## Instantiating a Circuit structure Template: the Region Graph

Initialize a _Quad Graph_ region graph.

In [ ]:
from cirkit.templates.region_graph import QuadTree
region_graph = QuadTree(shape=(height, height))

Others available region graphs are the _Random Binary Tree_ and the _Poon Domingos_, whose imports are showed below.

In [ ]:
from cirkit.templates.region_graph import RandomBinaryTree, PoonDomingos

## Constructing the Symbolic Circuit Representation

From the region graph definition above, we now construct the symbolic circuit representation. Note that this circuit representation is _not_ executable, i.e., you cannot do learn it or do inference with it. It will be compiled later, by choosing a backend such as torch.

To do so, we first define the factories that will be used to construct symbolic layers. Note that we choose the parameterization at the symbolic level. That is, we guarantee non-negative parameters by passing them through an exponential function. Moreover, we can choose how to parameterize the categorical distributions used to model the distribution of pixel values in the 0-255 range. In this case, we use a log softmax function. We choose to initialize the weights of the circuit by sampling from a normal distribution.

In [ ]:
from cirkit.utils.scope import Scope
from cirkit.symbolic.parameters import LogSoftmaxParameter, ExpParameter, Parameter
from cirkit.symbolic.layers import CategoricalLayer, DenseLayer, KroneckerLayer, MixingLayer
from cirkit.symbolic.initializers import NormalInitializer

In [ ]:
def categorical_layer_factory(
    scope: Scope,
    num_units: int,
    num_channels: int
) -> CategoricalLayer:
    return CategoricalLayer(
        scope, num_units, num_channels, num_categories=256,
        parameterization=lambda p: Parameter.from_unary(LogSoftmaxParameter(p.shape), p),
        initializer=NormalInitializer(0.0, 1e-2)
    )

def kronecker_layer_factory(
    scope: Scope, num_input_units: int, arity: int
) -> HadamardLayer:
    return KroneckerLayer(scope, num_input_units, arity)

def dense_layer_factory(
    scope: Scope,
    num_input_units: int,
    num_output_units: int
) -> DenseLayer:
    return DenseLayer(
        scope, num_input_units, num_output_units,
        parameterization=lambda p: Parameter.from_unary(ExpParameter(p.shape), p),
        initializer=NormalInitializer(0.0, 1e-1)
    )


def mixing_layer_factory(
    scope: Scope, num_units: int, arity: int
) -> MixingLayer:
    return MixingLayer(
        scope, num_units, arity,
        parameterization=lambda p: Parameter.from_unary(ExpParameter(p.shape), p),
        initializer=NormalInitializer(0.0, 1e-1)
    )

Then, we call a function to construct the symbolic circuit from region graph, by specifying the number of units and the factories to build layers.

In [ ]:
from cirkit.symbolic.circuit import Circuit

In [ ]:
symbolic_circuit = Circuit.from_region_graph(
    region_graph,
    num_input_units=8,
    num_sum_units=8,
    input_factory=categorical_layer_factory,
    sum_factory=dense_layer_factory,
    prod_factory=kronecker_layer_factory,
    mixing_factory=mixing_layer_factory
)

We can retrieve some information about the circuit and its structural properties as follows.

In [ ]:
print(f'Smooth: {symbolic_circuit.is_smooth}')
print(f'Decomposable: {symbolic_circuit.is_decomposable}')
print(f'Number of variables: {symbolic_circuit.num_variables}')
print(f'Number of channels per variable: {symbolic_circuit.num_channels}')

## Compiling the Symbolic Circuit

We are ready to compile the symbolic circuit constructed above into another one that we can learn and/or do inference. To do so, we have to choose a compilation backend. In this case, we choose torch as a backend.

In [ ]:
import torch
device = torch.device('cuda')  # The device to use
torch.manual_seed(42)
if 'cuda' in device.type:
    torch.cuda.manual_seed(42)

We first need to instantiate a circuit pipeline context and specify the backend to be used, as well as optional compilation flags, e.g., whether to fold the circuit or which inference semiring to use. Finally, we use the pipeline context to compile the symbolic circuit.

In [ ]:
from cirkit.pipeline import PipelineContext

In [ ]:
%%time
ctx = PipelineContext(
    backend='torch',   # Choose the torch compilation backend
    fold=True,         # Fold the circuit, this is a backend-specific compilation flag
    semiring='lse-sum' # Use the (R, +, *) semiring, where + is the log-sum-exp and * is the sum
)
circuit = ctx.compile(symbolic_circuit)

# Alternatively, one can use the Python _with_ statement to compile circuits inside of it.
#
# from cirkit.pipeline import compile
# with PipelineContext(backend='torch', fold=True, semiring='lse-sum') as ctx:
#    circuit = compile(symbolic_circuit)

Note that the compilation step, comprising the folding optimization, required about 1 second for a circuit with almost 5000 layers.

In [ ]:
print(len(list(symbolic_circuit.layers)))

We observe how the tensorized circuit has much fewer layers, as the great majority has been folded.

In [ ]:
print(circuit)

## Training and Testing

We are now ready to learn the parameters and do inference First, we wrap our data into PyTorch data loaders by specifying the batch size. Then, we initialize any PyTorch optimizer, e.g. SGD with momentum in this case.

In [ ]:
from torch import optim
from torch.utils.data import DataLoader
train_dataloader = DataLoader(data_train, shuffle=True, batch_size=256, drop_last=True, num_workers=4)
test_dataloader = DataLoader(data_test, shuffle=False, batch_size=256, num_workers=4)
optimizer = optim.SGD(circuit.parameters(), lr=0.1, momentum=0.95)

### Compiling the Circuit computing the Partition Function

Here, we choose to optimize the parameters by minimizing the negative log-likelihood. However, since the circuit is not already normalized (as we parameterized the sums using an exponential), we need to instantiate a circuit computing the partition function explicitly. That is, we integrate the circuit within the pipeline context as follows.

In [ ]:
pf_circuit = ctx.integrate(circuit)

# Alternatively, one can use the _with_ statement.
#
# from cirkit.pipeline import integrate
# with ctx:
#     pf_circuit = integrate(tensorized_circuit)

# Another way to perform the integration is by using the _symbolic functional_ APIs.
# This gives us additional flexibility by operating over circuits _before compiling them_.
#
# import cirkit.symbolic.functional as SF
# with ctx:
#     pf_symbolic_circuit = SF.integrate(symbolic_circuit)
#     pf_circuit = compile(pf_symbolic_circuit)

# The type of the circuit obtained by integration of all variables is a 'constant' circuit,
# i.e., it does not take inputs.
print(type(pf_circuit))

Note that the context will take care of the parameters being shared between the compiled circuit and its other version, which computes the partition function. Finally, we learn the parameters by minimizing the negative log-likelihood, similarly as any other model in torch.

In [ ]:
# Move circuits to device
circuit = circuit.to(device)
pf_circuit = pf_circuit.to(device)

In [ ]:
import time

In [ ]:
start_time = time.perf_counter()
num_epochs = 3
step_idx = 0
running_loss = 0.0
for epoch_idx in range(num_epochs):
    for i, (batch, _) in enumerate(train_dataloader):
        batch = batch.to(device).unsqueeze(dim=1)   # Add a channel dimension
        log_output = circuit(batch)                 # Compute the log output of the circuit
        log_pf = pf_circuit()                       # Compute the log partition function of the circuit
        lls = log_output - log_pf                   # Compute the log-likelihood
        loss = -torch.mean(lls)   # The loss is the negative average log-likelihood
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.detach() * len(batch)
        step_idx += 1
        if step_idx % 100 == 0:
            print(f"Step {step_idx}: Average NLL: {running_loss / (100 * len(batch)):.3f}")
            running_loss = 0.0
end_time = time.perf_counter()
print(f"Training time: {end_time - start_time} seconds")

We then evaluate our model on test data by computing the average log-likelihood and bits per dimension.

In [ ]:
circuit.eval()
pf_circuit.eval()

with torch.no_grad():
    test_lls = 0.0
    log_pf = pf_circuit()  # Compute the log partition function of the circuit (just once as we are evaluating)
    for batch, _ in test_dataloader:
        batch = batch.to(device).unsqueeze(dim=1)   # Add a channel dimension
        log_output = circuit(batch)                 # Compute the log output of the circuit
        lls = log_output - log_pf                   # Compute the log-likelihood
        test_lls += lls.sum().item()
    average_ll = test_lls / len(data_test)
    bpd = -average_ll / (num_variables * np.log(2.0))
    print(f"Average test LL: {average_ll:.3f}")
    print(f"Bits per dimension: {bpd}")

# Bonus: Optimize your Circuit

The circuit that we learned above contains expensive kronecker product layers that can be optimized by fusing them with the dense layers.
In the following, we compile the circuit by activating the optimize flag of the compiler.
Based on the layers being used, this will often yield a different torch circuit that is more time and memory efficient.

In [ ]:
%%time
ctx = PipelineContext(
    backend='torch',   # Choose the torch compilation backend
    fold=True,         # Fold the circuit, this is a backend-specific compilation flag
    optimize=True,     # Active the optimizations of the torch compilatio backend
    semiring='lse-sum' # Use the (R, +, *) semiring, where + is the log-sum-exp and * is the sum
)
circuit = ctx.compile(symbolic_circuit)

Note that the optimized circuit will now have the Tucker layer, which is a fused layer combining the kronecker and dense layers that is much more efficient.

In [ ]:
print(circuit)

We learn again the parameters of the circuit using the same code above.

In [ ]:
from torch import optim
from torch.utils.data import DataLoader
train_dataloader = DataLoader(data_train, shuffle=True, batch_size=256, drop_last=True, num_workers=4)
test_dataloader = DataLoader(data_test, shuffle=False, batch_size=256, num_workers=4)
optimizer = optim.SGD(circuit.parameters(), lr=0.1, momentum=0.95)

In [ ]:
import time

In [ ]:
start_time = time.perf_counter()
num_epochs = 3
step_idx = 0
running_loss = 0.0
for epoch_idx in range(num_epochs):
    for i, (batch, _) in enumerate(train_dataloader):
        batch = batch.to(device).unsqueeze(dim=1)   # Add a channel dimension
        log_output = circuit(batch)                 # Compute the log output of the circuit
        log_pf = pf_circuit()                       # Compute the log partition function of the circuit
        lls = log_output - log_pf                   # Compute the log-likelihood
        loss = -torch.mean(lls)   # The loss is the negative average log-likelihood
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.detach() * len(batch)
        step_idx += 1
        if step_idx % 100 == 0:
            print(f"Step {step_idx}: Average NLL: {running_loss / (100 * len(batch)):.3f}")
            running_loss = 0.0
end_time = time.perf_counter()
print(f"Training time: {end_time - start_time} seconds")

We observe a reduced training time given by the optimizations.